In [1]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [3]:
from dotenv import load_dotenv
import os


# Load the .env file
load_dotenv("/content/.env")

# Now fetch your connection string
conn_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
print("Connection String Loaded:", conn_string[:50] + "...")

Connection String Loaded: DefaultEndpointsProtocol=https;AccountName=hrattri...


In [4]:
from dotenv import load_dotenv
load_dotenv("/content/.env")

True

In [5]:
pip install azure-storage-blob # This is Microsoft’s official Python SDK to work with Azure Blob Storage.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 17.5 MB/s eta 0:00:00



The key class here is BlobServiceClient, which lets you:

Connect to your storage account.

Upload/download files to containers.

List blobs (files) inside containers.

Delete or update blobs.

In [ ]:
# --- Step 1: Connect to Azure Blob and list files in raw/ container ---

import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

# Load environment variables from .env file
load_dotenv("/content/.env")

# Read connection string
conn_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
if conn_string is None:
    raise ValueError("Connection string not found. Please check your .env file.")

# Create Blob service client
blob_service_client = BlobServiceClient.from_connection_string(conn_string)

# Choose the raw container
raw_container = "raw"

# List blobs inside the raw container
print("files in raw container")

container_client = blob_service_client.get_container_client(raw_container)
for blob in container_client.list_blobs():
 print("-", blob.name)

files in raw container
- IBM.csv


## 📥 Downloading IBM.csv from Azure Blob into Colab so we can clean it.

In [ ]:
# --- Step 2: Download raw file from Azure Blob to Colab ---

raw_file_name = "IBM.csv"


# Local path where file will be saved in Colab
download_path = f"./{raw_file_name}"

# Get blob client
blob_client = blob_service_client.get_blob_client(container=raw_container, blob=raw_file_name)

# Download file
with open(download_path,"wb") as f:
  data = blob_client.download_blob()
  f.write(data.readall())

print(f" downloaded {raw_file_name} to {download_path}")

 downloaded IBM.csv to ./IBM.csv


In [ ]:
import pandas as pd

df = pd.read_csv("IBM.csv")

print(df.head())

   Age Attrition              Department  DistanceFromHome  Education  \
0   41       Yes                   Sales                 1          2   
1   49        No  Research & Development                 8          1   
2   37       Yes  Research & Development                 2          2   
3   33        No  Research & Development                 3          4   
4   27        No  Research & Development                 2          1   

  EducationField  EnvironmentSatisfaction  JobSatisfaction MaritalStatus  \
0  Life Sciences                        2                4        Single   
1  Life Sciences                        3                2       Married   
2          Other                        4                3        Single   
3  Life Sciences                        4                3       Married   
4        Medical                        1                2       Married   

   MonthlyIncome  NumCompaniesWorked  WorkLifeBalance  YearsAtCompany  
0           5993                

In [ ]:
# Generate EmployeeNumber starting from 1 up to number of rows
df.insert(0, "EmployeeNumber", range(1, len(df) + 1))


In [ ]:
print(df.head())

   EmployeeNumber  Age Attrition              Department  DistanceFromHome  \
0               1   41       Yes                   Sales                 1   
1               2   49        No  Research & Development                 8   
2               3   37       Yes  Research & Development                 2   
3               4   33        No  Research & Development                 3   
4               5   27        No  Research & Development                 2   

   Education EducationField  EnvironmentSatisfaction  JobSatisfaction  \
0          2  Life Sciences                        2                4   
1          1  Life Sciences                        3                2   
2          2          Other                        4                3   
3          4  Life Sciences                        4                3   
4          1        Medical                        1                2   

  MaritalStatus  MonthlyIncome  NumCompaniesWorked  WorkLifeBalance  \
0        Single      

In [ ]:
# Define all mappings
mappings = {
    "Education": {
        1: "Below College",
        2: "College",
        3: "Bachelor",
        4: "Master",
        5: "Doctor"
    },
    "EnvironmentSatisfaction": {
        1: "Low",
        2: "Medium",
        3: "High",
        4: "Very High"
    },
    "JobInvolvement": {
        1: "Low",
        2: "Medium",
        3: "High",
        4: "Very High"
    },
    "JobSatisfaction": {
        1: "Low",
        2: "Medium",
        3: "High",
        4: "Very High"
    },
    "PerformanceRating": {
        1: "Low",
        2: "Good",
        3: "Excellent",
        4: "Outstanding"
    },
    "RelationshipSatisfaction": {
        1: "Low",
        2: "Medium",
        3: "High",
        4: "Very High"
    },
    "WorkLifeBalance": {
        1: "Bad",
        2: "Good",
        3: "Better",
        4: "Best"
    }
}


In [ ]:
import json

# Save dictionary to JSON file
with open("mappings.json", "w") as f:
    json.dump(mappings, f, indent=4)

In [ ]:
!pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=f5429425c16fdac55989e403f5501367a60dac7da355d3094731ab527460712f
  Stored in directory: /root/.cache/pip/wheels/5f/d4/d7/4189b07b5902ee9f3ce0dbb14909fbe8037c39d6c63ffd49c9
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      

In [ ]:
from ydata_profiling import ProfileReport

# Load dataset
df = pd.read_csv("IBM.csv")

# create profile report
profile = ProfileReport(df, title="IBM HR Data Profiling Report", explorative=True)

profile.to_file("IBM_HR_Profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 13/13 [00:00<00:00, 36.84it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from google.colab import files
files.download("IBM_HR_Profile.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

no misisng values found so far and duplicate columns

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Ensure logs folder exists (safe for every run)
os.makedirs("logs", exist_ok=True)

# Load raw data
df = pd.read_csv("IBM.csv")


# Logging info
log = {
    "file_name": "IBM.csv",
    "rows": len(df),
    "nulls": df.isnull().sum().to_dict(),
    "duplicates": df.duplicated().sum(),
    "outliers": {}
}

# Handle duplicates
df = df.drop_duplicates()

# Handle missing values
df = df.fillna({
    "MonthlyIncome": df["MonthlyIncome"].median(),
    "WorkLifeBalance": df["WorkLifeBalance"].mode()[0]
})

# Outlier detection for Age only (business rule: valid ages 18–62)
outliers_age = df[(df["Age"] < 18) | (df["Age"] > 62)]
log["outliers"]["Age"] = len(outliers_age)
df["Age"] = np.clip(df["Age"], 18, 62)  # winsorize Age only

# No winsorization for MonthlyIncome
log["outliers"]["MonthlyIncome"] = "Skipped (kept original values)"

# ✅ Ensure EmployeeNumber exists
if "EmployeeNumber" not in df.columns:
    # Generate new IDs if missing
    df.insert(0, "EmployeeNumber", range(1, len(df) + 1))

# Remapping the values for dashboard
for col, mapping in mappings.items():
  if col in df.columns:
    df[col] = df[col].replace(mapping)

# Save cleaned data
file_name = f"IBM_cleaned_{datetime.today().strftime('%Y%m%d')}.csv"
df.to_csv(file_name, index=False)

# Save log
log_df = pd.DataFrame([log])
log_df.to_csv("logs/validation_log.csv", mode="a", header=False, index=False)

# Print results
print("Rows after cleaning:", len(df))
print("Nulls handled:", log["nulls"])
print("Duplicates removed:", log["duplicates"])
print("Outliers log:", log["outliers"])
print(f"Cleaned file saved as {file_name}")


Rows after cleaning: 1470
Nulls handled: {'Age': 0, 'Attrition': 0, 'Department': 0, 'DistanceFromHome': 0, 'Education': 0, 'EducationField': 0, 'EnvironmentSatisfaction': 0, 'JobSatisfaction': 0, 'MaritalStatus': 0, 'MonthlyIncome': 0, 'NumCompaniesWorked': 0, 'WorkLifeBalance': 0, 'YearsAtCompany': 0}
Duplicates removed: 0
Outliers log: {'Age': 0, 'MonthlyIncome': 'Skipped (kept original values)'}
Cleaned file saved as IBM_cleaned_20250824.csv


# Upload Clean Data to Azure Blob (Clean Container)

In [ ]:
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

# Load environment variables from .env file
load_dotenv("/content/.env")

True

In [ ]:
# Azure connection

conn_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
if conn_string is None:
    raise ValueError("Connection string not found. Please check your .env file.")

# Create Blob service client
blob_service_client = BlobServiceClient.from_connection_string(conn_string)

# clean file name
clean_file = file_name

# Upload CSV to "clean" container
claen_container = "clean"
blob_client = blob_service_client.get_blob_client(container=claen_container, blob=clean_file)

with open(clean_file, "rb") as data1:
    blob_client.upload_blob(data1, overwrite=True)

# List blobs inside the clean container
print("files in clean container")

files in clean container


In [ ]:
import json

# Save log as JSON file
log_file = f"log_{datetime.today().strftime('%Y%m%d_%H%M%S')}.json"
with open(log_file, "w") as f:
    json.dump(log, f, indent=4, default=str)

# Upload JSON log to "logs" container
log_blob_client = blob_service_client.get_blob_client(container="logs", blob=log_file)
with open(log_file, "rb") as data:
    log_blob_client.upload_blob(data, overwrite=True)

print(f"✅ Log uploaded to logs/{log_file}")


✅ Log uploaded to logs/log_20250824_134717.json


In [ ]:
ad = pd.read_csv("IBM_cleaned_20250824.csv")

In [ ]:
ad.head()

,EmployeeNumber,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany
0,1,41,Yes,Sales,1,College,Life Sciences,Medium,Very High,Single,5993,8,Bad,6
1,2,49,No,Research & Development,8,Below College,Life Sciences,High,Medium,Married,5130,1,Better,10
2,3,37,Yes,Research & Development,2,College,Other,Very High,High,Single,2090,6,Better,0
3,4,33,No,Research & Development,3,Master,Life Sciences,Very High,High,Married,2909,1,Better,8
4,5,27,No,Research & Development,2,Below College,Medical,Low,Medium,Married,3468,9,Better,2
